cvxlab - ROLLING HORIZON TEST

In [1]:

from data_handler import *
import cvxlab as cl

H_group = {
    1: range(2000, 2030),
    2: range(2005, 2030),
    3: range(2010, 2030),
    4: range(2015, 2030),
    5: range(2020, 2030)
}

main_dir_path = 'C:/Users/baioc/OneDrive - Politecnico di Milano/PhD/CVXLAB/Concepts/Rolling'
fh_len = 5
delta = 3




RH1: 2000 - 2030

In [2]:
H=1

model_dir_name = f'H{H}'


pass_model_settings(1, main_dir_path, start = True)
update_sets(1, H_group[1], main_dir_path, delta,start=True)
update_input_data(1, H_group[1], main_dir_path, delta, fh_len, start=True)

# #define model instance
# model = cl.Model(
#     model_dir_name=model_dir_name,
#     main_dir_path=main_dir_path,
#     log_level='debug',
#     model_settings_from='xlsx',
#     multiple_input_files=True,
#     use_existing_data=False,
#     detailed_validation=True,
# )

Copying model_settings.xlsx to horizon H1
Updating sets.xlsx to horizon H1
Processing av_h_max.xlsx...
Processing av_h_min.xlsx...
Processing av_y_max.xlsx...
Processing av_y_min.xlsx...
Processing cap_max.xlsx...
Processing cap_max_y.xlsx...
Processing cap_min.xlsx...
Processing cap_min_y.xlsx...
Processing cap_plan_ex.xlsx...
Processing cap_start.xlsx...
Processing co.xlsx...
Processing c_fix.xlsx...
Processing c_inv.xlsx...
Processing c_var.xlsx...
Processing dc.xlsx...
Processing dp.xlsx...
Processing dr.xlsx...
Processing Iff_h.xlsx...
Processing I_ff.xlsx...
Processing lt_new.xlsx...
Processing lt_res.xlsx...
Processing sf.xlsx...
Processing u.xlsx...
Processing v.xlsx...
Processing Y.xlsx...


c:\Users\baioc\Documents\GitHub\pyesm\data_handler.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['flows_agg_Name']=='Energia elettrica'][ 'values'] = demand_update


In [ ]:
import pandas as pd

demand = pd.read_excel('C:/Users/baioc/OneDrive - Politecnico di Milano/PhD/CVXLAB/Concepts/Rolling/defaults/demand.xlsx')
demand



In [ ]:

#load data
model.load_model_coordinates()
model.initialize_blank_data_structure()
model.load_exogenous_data_to_sqlite_database(force_overwrite=True)

#initialize model and run
model.initialize_problems()
model.run_model(
    verbose=False,
    solver='GUROBI',
    integrated_problems=False,
)

#save results
model.load_results_to_database(force_overwrite=True)

In [ ]:
df = model.variable(name='Cap_plan').reset_index().rename(columns={'index': 'year'})
df = df.melt(id_vars=['year'], value_name='cap_plan')
upper_bound = min(H_group[2]) + delta + fh_len 
df =df.loc[df['year']< upper_bound]
df['year'].unique()

In [ ]:
vals = get_planned_cap(model, delta, fh_len, H_group[1])
vals.rename(columns={'year': 'years_Name','variable':'techs_Name', 'cap_plan_ex': 'values'}, inplace=True)
vals

In [ ]:
import pandas as pd
df = pd.read_excel('C:/Users/baioc/OneDrive - Politecnico di Milano/PhD/CVXLAB/Concepts/Rolling/H2/input_data/cap_plan_ex.xlsx')

df.update(vals)


In [ ]:
model.variable(name='c_inv', sub_problem_key=0).loc[:,'Geotermico'][0] * model.variable(name='Cap_built').loc[:,'Geotermico'].sum()

In [ ]:
print(model.variable(name='c_fix_tot').loc[:,'Geotermico'].sum())
print(model.variable(name='c_fix', sub_problem_key=0).loc[:,'Geotermico'][0] * model.variable(name='Cap_op').loc[:,'Geotermico'].sum())

In [ ]:
print(model.variable(name='c_var_tot').loc['Geotermico',:].sum()/1e10)
print(model.variable(name='c_var', sub_problem_key=0).loc['Geotermico']*  model.variable(name='X').loc[:, 'Geotermico'].sum())
#   *)

In [ ]:
power = ['Coal Power Plant', 'Eolico', 'CCGT', 'Geotermico', 'Idrico', 'OCGT',
       'Impianto ad Olio combustibile', 'PV', 'Impianto a BioGas',
       'Impianto a Biomassa']

model.variable(name='X').loc[:, power].sum(axis=1)

In [ ]:
Qc = model.variable(name='Q').loc[:,'Energia elettrica - Coal']
Qc_h = model.variable(name='Qh', sub_problem_key=0).loc[:,'Energia elettrica - Coal'].sum()*365
print(Qc, Qc_h)

In [ ]:
Xc = model.variable(name='X').loc[:,'Coal Power Plant']
Xc_h = model.variable(name='Xh', sub_problem_key=0).loc[:,'Coal Power Plant'].sum()*365
print(Qc, Qc_h)

- - - 
SAVING/LOADING MODEL INSTANCE

In case model instance generation is taking huge time, it is possible to save/load model instance to avoid regenerating it several times.

Notice that - for unknown reasons - model instance can be loaded only if it has not already solve (i.e. do not save model instance if you run the model!)

In [ ]:
model.variable(name='Qh_agg', sub_problem_key=0).sum().sum()

In [ ]:
model.variable(name='Q', sub_problem_key=0).loc[2000,
                              ['Energia elettrica - Coal',
       'Energia elettrica - Eolico', 'Energia elettrica - Gas Naturale',
       'Energia elettrica - Geotermico', 'Energia elettrica - Idrico',
       'Energia elettrica - Nucleare', 'Energia elettrica - Olio combustibile',
       'Energia elettrica - PV', 'Energia elettrica - BioGas',
       'Energia elettrica - Biomassa']].sum().sum()

In [ ]:
model.variable(name='Q').loc[:,'Energia elettrica - Coal']

In [ ]:
cl.handle_model_instance(
    action='save',
    instance=model,
    file_name='instance_name'
)

In [ ]:
model = cl.handle_model_instance(
    action='load'
    file_name='file_name',
    source_dir_path=r"D:/git_repos/pyesm/tests/models/integrated/1_coupled_model/instances",
)

- - -
DATA and SYMBOLIC PROBLEM UPDATE without regenerating Model instance

- initialize_problems(): if symbolic problem has modified, upload it and generates a new numerical model

- update_database_and_problem(): in case of modifications in input data files (but not in sets, nor in variables structures) and symbolic problem, update database and problem

- reinitialize_sqlite_database(): reinitialize sqlite database endogenous variables


In [ ]:
# in case of need for generating only one or more data input files/tabs
model.generate_input_data_files(table_key_list=[])

In [ ]:
# in case of modifications in symbolic problem only,
# update problems dataframe and symbolic problem
model.initialize_problems()

In [ ]:
# in case of modifications in input data files (but not in sets, nor in 
# variables structures) and symbolic problem, update database and problem
model.update_database_and_problem()

In [ ]:
# in case of modifications in input data files (but not in sets, nor in 
# variables structures) and symbolic problem, reinitialize sqlite database for
# endogenous variables
model.reinitialize_sqlite_database(force_overwrite=True)

- - -
SOLVE NUMERICAL PROBLEM and ENDOGENOUS DATA EXPORT

- run_model(): run model with various settings
- load_results_to_database(): once model has successfully solved, load endogenous parameters data to sqlite database.
- check_model_results(): check model results compared to an existing database (path and name of the database with the expected results in model settings attributes)

In [ ]:
model.run_model(
    verbose=False,
    solver='GUROBI',
    integrated_problems=True,
)

In [ ]:
model.load_results_to_database(force_overwrite=True)

In [ ]:
model.check_model_results()

- - -
UTILITIES

- sets(): list of model sets
- variables(): dictionary with variable name (key) and shape of teh variable (value)
- variable(name, ...): visual check of variables content in the database
- set(name): visual check of sets content in the database

In [ ]:
model.variables
model.sets

In [ ]:
model.set('products')

In [ ]:
model.variable(name='L',)